# Steps to Use the Notebook:
## 1.Open the Notebook
*   Navigate to the provided Python notebook, titled something like download_models.
*   Ensure you're in the correct kernel (py310env for Python 3.10) before running the notebook.
## 2.Run the Notebook


*   Once the notebook is open, you'll see pre-configured cells for downloading Stable Diffusion models, LoRA, and VAE.
*   Simply execute each cell by selecting it and clicking the Run button (or pressing Shift + Enter).



# Before Runing Clear .cache

In [ ]:
!rm -rf /home/studio-lab-user/.cache
!df -h | grep -E 'Avail|home'

print('[1;32mDone!')

# Models or Checkpoint


In [ ]:
import requests
import time
import os
import subprocess

# Download path
download_path = "/home/studio-lab-user/stable-diffusion-webui/models/Stable-diffusion/"

# Ensure the download path exists, create it if it doesn't
os.makedirs(download_path, exist_ok=True)

# Helper function to convert bytes into human-readable format
def format_size(size):
    for unit in ['B', 'KB', 'MB', 'GB', 'TB']:
        if size < 1024:
            return f"{size:.2f} {unit}"
        size /= 1024

# Function to check available storage space
def check_storage_space():
    result = subprocess.run(["df", "-h", "--output=avail,target"], capture_output=True, text=True)
    lines = result.stdout.splitlines()

    for line in lines:
        if '/home' in line:
            available_space = line.split()[0]  # Get available space value
            return available_space
    return "0M"  # Default value if no storage info available

# Function to download model with reduced output frequency
def download_model(url, filename, headers):
    try:
        # Automatically handle the file extension based on the filename provided
        response = requests.get(url, headers=headers, stream=True)

        if response.status_code == 200:
            total_size = int(response.headers.get('content-length', 0))
            downloaded_size = 0
            start_time = time.time()

            with open(filename, 'wb') as file:
                chunk_size = 1024 * 1024  # Reduce the chunk size
                for chunk in response.iter_content(chunk_size=chunk_size):
                    if chunk:
                        try:
                            file.write(chunk)
                            downloaded_size += len(chunk)

                            if downloaded_size >= 10 * 1024 * 1024:  # Log progress every 10 MB
                                elapsed_time = time.time() - start_time
                                speed = (downloaded_size / (1024 * 1024)) / elapsed_time
                                print(f"Downloaded {format_size(downloaded_size)} of {format_size(total_size)} at {speed:.2f} MB/s", end='\r')
                        except OSError as e:
                            if "No space left on device" in str(e):
                                print(f"\n\nError: No space left on device while downloading {filename}.")
                                return "no_space"  # Return a specific message if no space
                            raise
            print(f"\n\nDownloaded {filename} successfully!\n")
            return "success"
        else:
            print(f"\n\nFailed to download {filename}. Status code: {response.status_code}, Message: {response.text}\n")
            return "error"
    except Exception as e:
        print(f"\n\nError during download of {filename}: {e}\n")
        if "data exceeds" in str(e).lower():
            print("Error: Data exceeds Jupyter memory limit. Please download outside Jupyter Lab.\n")
        return "error"

# Combined function to handle both Hugging Face and CivitAI links
def download_models(model_list, huggingface_headers, civitai_headers):
    for model in model_list:
        if model["enabled"]:
            url = model["link"]
            filename = os.path.join(download_path, model["filename"])

            # Identify if it's Hugging Face or CivitAI by checking the URL structure
            if "huggingface.co" in url:
                headers = huggingface_headers
                platform = "Hugging Face"
            elif "civitai.com" in url:
                headers = civitai_headers
                platform = "CivitAI"
            else:
                print(f"\n\nUnknown platform for {filename}. Skipping...\n")
                continue

            print(f"\nStarting download from {platform} for {filename}...\n")

            # Check storage before downloading
            available_space = check_storage_space()
            print(f"Available space before downloading: {available_space}\n")

            if available_space.endswith('G') or available_space.endswith('M'):
                result = download_model(url, filename, headers)
                if result == "no_space":
                    print(f"\n\nNo space available. Stopping remaining downloads.\n")
                    break
            else:
                print(f"\n\nNo space available for {filename}. Skipping...\n")
        else:
            print(f"\n\nSkipping {model['filename']} (Disabled)\n")




# ----------------------------------------
# Example combined model inputs (both Hugging Face and CivitAI)
model_list = [

    {"link": "url_link_1",
     "filename": "model1.safetensors",     "enabled": False  },

    {"link": "url_link_2",
     "filename": "model2.safetensors",    "enabled": False  },

    {"link": "url_link_3",
     "filename": "model3.safetensors",    "enabled": False  },

    {"link": "url_link_4",
     "filename": "model4",    "enabled": False  }


]


# ----------------------------------------
# Token Section (define them here and create headers below)
huggingface_token = "your_huggingface_token"
civitai_api_key = "your_civitai_token"



# Now, after tokens are defined, create the headers
huggingface_headers = {
    "Authorization": f"Bearer {huggingface_token}"
}
civitai_headers = {
    "Authorization": f"Bearer {civitai_api_key}"
}

# Call the combined download function
download_models(model_list, huggingface_headers, civitai_headers)

# Clear cache after downloads
print("\n\nClearing cache...\n")
subprocess.run(["rm", "-rf", ".cache"])

# Final storage check
print("\nAll downloads done!")
print(f"Available space after downloading: {check_storage_space()}\n")


# LoRA

In [ ]:
import requests
import time
import os
import subprocess

# Download path
download_path = "/home/studio-lab-user/stable-diffusion-webui/models/Lora/"

# Ensure the download path exists, create it if it doesn't
os.makedirs(download_path, exist_ok=True)

# Helper function to convert bytes into human-readable format
def format_size(size):
    for unit in ['B', 'KB', 'MB', 'GB', 'TB']:
        if size < 1024:
            return f"{size:.2f} {unit}"
        size /= 1024

# Function to check available storage space
def check_storage_space():
    result = subprocess.run(["df", "-h", "--output=avail,target"], capture_output=True, text=True)
    lines = result.stdout.splitlines()

    for line in lines:
        if '/home' in line:
            available_space = line.split()[0]  # Get available space value
            return available_space
    return "0M"  # Default value if no storage info available

# Function to download model with reduced output frequency
def download_model(url, filename, headers):
    try:
        # Automatically handle the file extension based on the filename provided
        response = requests.get(url, headers=headers, stream=True)

        if response.status_code == 200:
            total_size = int(response.headers.get('content-length', 0))
            downloaded_size = 0
            start_time = time.time()

            with open(filename, 'wb') as file:
                chunk_size = 1024 * 1024  # Reduce the chunk size
                for chunk in response.iter_content(chunk_size=chunk_size):
                    if chunk:
                        try:
                            file.write(chunk)
                            downloaded_size += len(chunk)

                            if downloaded_size >= 10 * 1024 * 1024:  # Log progress every 10 MB
                                elapsed_time = time.time() - start_time
                                speed = (downloaded_size / (1024 * 1024)) / elapsed_time
                                print(f"Downloaded {format_size(downloaded_size)} of {format_size(total_size)} at {speed:.2f} MB/s", end='\r')
                        except OSError as e:
                            if "No space left on device" in str(e):
                                print(f"\n\nError: No space left on device while downloading {filename}.")
                                return "no_space"  # Return a specific message if no space
                            raise
            print(f"\n\nDownloaded {filename} successfully!\n")
            return "success"
        else:
            print(f"\n\nFailed to download {filename}. Status code: {response.status_code}, Message: {response.text}\n")
            return "error"
    except Exception as e:
        print(f"\n\nError during download of {filename}: {e}\n")
        if "data exceeds" in str(e).lower():
            print("Error: Data exceeds Jupyter memory limit. Please download outside Jupyter Lab.\n")
        return "error"

# Combined function to handle both Hugging Face and CivitAI links
def download_models(model_list, huggingface_headers, civitai_headers):
    for model in model_list:
        if model["enabled"]:
            url = model["link"]
            filename = os.path.join(download_path, model["filename"])

            # Identify if it's Hugging Face or CivitAI by checking the URL structure
            if "huggingface.co" in url:
                headers = huggingface_headers
                platform = "Hugging Face"
            elif "civitai.com" in url:
                headers = civitai_headers
                platform = "CivitAI"
            else:
                print(f"\n\nUnknown platform for {filename}. Skipping...\n")
                continue

            print(f"\nStarting download from {platform} for {filename}...\n")

            # Check storage before downloading
            available_space = check_storage_space()
            print(f"Available space before downloading: {available_space}\n")

            if available_space.endswith('G') or available_space.endswith('M'):
                result = download_model(url, filename, headers)
                if result == "no_space":
                    print(f"\n\nNo space available. Stopping remaining downloads.\n")
                    break
            else:
                print(f"\n\nNo space available for {filename}. Skipping...\n")
        else:
            print(f"\n\nSkipping {model['filename']} (Disabled)\n")


# ----------------------------------------
# Example combined model inputs (both Hugging Face and CivitAI)
model_list = [

    {"link": "url_link_1",
     "filename": "model1.safetensors",     "enabled": False  },

    {"link": "url_link_2",
     "filename": "model2.safetensors",    "enabled": False  },

    {"link": "url_link_3",
     "filename": "model3.safetensors",    "enabled": False  },

    {"link": "url_link_4",
     "filename": "model4",    "enabled": False  }

]

# ----------------------------------------
# Token Section (define them here and create headers below)
huggingface_token = "your_huggingface_token"
civitai_api_key = "your_civitai_token"

# Now, after tokens are defined, create the headers
huggingface_headers = {
    "Authorization": f"Bearer {huggingface_token}"
}
civitai_headers = {
    "Authorization": f"Bearer {civitai_api_key}"
}

# Call the combined download function
download_models(model_list, huggingface_headers, civitai_headers)

# Clear cache after downloads
print("\n\nClearing cache...\n")
subprocess.run(["rm", "-rf", ".cache"])

# Final storage check
print("\nAll downloads done!")
print(f"Available space after downloading: {check_storage_space()}\n")


# VAE

In [ ]:
import requests
import time
import os
import subprocess

# Download path
download_path = "/home/studio-lab-user/stable-diffusion-webui/models/VAE/"

# Ensure the download path exists, create it if it doesn't
os.makedirs(download_path, exist_ok=True)

# Helper function to convert bytes into human-readable format
def format_size(size):
    for unit in ['B', 'KB', 'MB', 'GB', 'TB']:
        if size < 1024:
            return f"{size:.2f} {unit}"
        size /= 1024

# Function to check available storage space
def check_storage_space():
    result = subprocess.run(["df", "-h", "--output=avail,target"], capture_output=True, text=True)
    lines = result.stdout.splitlines()

    for line in lines:
        if '/home' in line:
            available_space = line.split()[0]  # Get available space value
            return available_space
    return "0M"  # Default value if no storage info available

# Function to download model with reduced output frequency
def download_model(url, filename, headers):
    try:
        # Automatically handle the file extension based on the filename provided
        response = requests.get(url, headers=headers, stream=True)

        if response.status_code == 200:
            total_size = int(response.headers.get('content-length', 0))
            downloaded_size = 0
            start_time = time.time()

            with open(filename, 'wb') as file:
                chunk_size = 1024 * 1024  # Reduce the chunk size
                for chunk in response.iter_content(chunk_size=chunk_size):
                    if chunk:
                        try:
                            file.write(chunk)
                            downloaded_size += len(chunk)

                            if downloaded_size >= 10 * 1024 * 1024:  # Log progress every 10 MB
                                elapsed_time = time.time() - start_time
                                speed = (downloaded_size / (1024 * 1024)) / elapsed_time
                                print(f"Downloaded {format_size(downloaded_size)} of {format_size(total_size)} at {speed:.2f} MB/s", end='\r')
                        except OSError as e:
                            if "No space left on device" in str(e):
                                print(f"\n\nError: No space left on device while downloading {filename}.")
                                return "no_space"  # Return a specific message if no space
                            raise
            print(f"\n\nDownloaded {filename} successfully!\n")
            return "success"
        else:
            print(f"\n\nFailed to download {filename}. Status code: {response.status_code}, Message: {response.text}\n")
            return "error"
    except Exception as e:
        print(f"\n\nError during download of {filename}: {e}\n")
        if "data exceeds" in str(e).lower():
            print("Error: Data exceeds Jupyter memory limit. Please download outside Jupyter Lab.\n")
        return "error"

# Combined function to handle both Hugging Face and CivitAI links
def download_models(model_list, huggingface_headers, civitai_headers):
    for model in model_list:
        if model["enabled"]:
            url = model["link"]
            filename = os.path.join(download_path, model["filename"])

            # Identify if it's Hugging Face or CivitAI by checking the URL structure
            if "huggingface.co" in url:
                headers = huggingface_headers
                platform = "Hugging Face"
            elif "civitai.com" in url:
                headers = civitai_headers
                platform = "CivitAI"
            else:
                print(f"\n\nUnknown platform for {filename}. Skipping...\n")
                continue

            print(f"\nStarting download from {platform} for {filename}...\n")

            # Check storage before downloading
            available_space = check_storage_space()
            print(f"Available space before downloading: {available_space}\n")

            if available_space.endswith('G') or available_space.endswith('M'):
                result = download_model(url, filename, headers)
                if result == "no_space":
                    print(f"\n\nNo space available. Stopping remaining downloads.\n")
                    break
            else:
                print(f"\n\nNo space available for {filename}. Skipping...\n")
        else:
            print(f"\n\nSkipping {model['filename']} (Disabled)\n")


# ----------------------------------------
# Example combined model inputs (both Hugging Face and CivitAI)
model_list = [

    {"link": "url_link_1",
     "filename": "model1.safetensors",     "enabled": False  },

    {"link": "url_link_2",
     "filename": "model2.safetensors",    "enabled": False  },

    {"link": "url_link_3",
     "filename": "model3.safetensors",    "enabled": False  },

    {"link": "url_link_4",
     "filename": "model4",    "enabled": False  }

]

# ----------------------------------------
# Token Section (define them here and create headers below)
huggingface_token = "your_huggingface_token"
civitai_api_key = "your_civitai_token"

# Now, after tokens are defined, create the headers
huggingface_headers = {
    "Authorization": f"Bearer {huggingface_token}"
}
civitai_headers = {
    "Authorization": f"Bearer {civitai_api_key}"
}

# Call the combined download function
download_models(model_list, huggingface_headers, civitai_headers)

# Clear cache after downloads
print("\n\nClearing cache...\n")
subprocess.run(["rm", "-rf", ".cache"])

# Final storage check
print("\nAll downloads done!")
print(f"Available space after downloading: {check_storage_space()}\n")
